In [1]:
from IPython.display import HTML

In [2]:
import requests


In [3]:
res = requests.get('http://project.initiumlab.com/news2video/case1/index.html')

In [4]:
html = res.content.decode('utf-8')

In [5]:
from readability.readability import Document
readable_article = Document(html).summary()
readable_title = Document(html).short_title()

In [6]:
readable_article

'<html><body><div><article id="main">\n\n  <h1>Walk: A Boring Story</h1>\n\n  I\'m going to tell you a boring story.\n\n  <img src="1.jpg" alt=""/>\n  <p>\n    This is the first step I move.\n  </p>\n\n  <img src="2.jpg" alt=""/>\n  <p>\n    This is the second step I move.\n  </p>\n\n  <img src="3.jpg" alt=""/>\n  <p>\n    This is the third step I move.\n  </p>\n\n  <img src="4.jpg" alt=""/>\n  <p>\n    This is the fourth step I move.\n  </p>\n\n  <img src="5.jpg" alt=""/>\n  <p>\n    This is the fifth step I move.\n  </p>\n\n  <img src="6.jpg" alt=""/>\n  <p>\n    This is the sixth step I move.\n  </p>\n\n  <img src="7.jpg" alt=""/>\n  <p>\n    This is the seventh step I move.\n  </p>\n\n  <img src="8.jpg" alt=""/>\n  <p>\n    This is the eighth step I move.\n  </p>\n\n  <img src="9.jpg" alt=""/>\n  <p>\n    This is the final step I move.\n  </p>\n\n  <p>\n    Thank you for reading this story.\n  </p>\n\n</article>\n\n</div></body></html>'

In [7]:
HTML(readable_article)

In [8]:
from os import path
base_url = path.dirname(res.request.url)
print(res.request.url)
print(base_url)

http://project.initiumlab.com/news2video/case1/index.html
http://project.initiumlab.com/news2video/case1


In [9]:
%%file test.txt
test

Overwriting test.txt


In [10]:
import html2text
print(html2text.html2text(readable_article))

# Walk: A Boring Story

I'm going to tell you a boring story. ![](1.jpg)

This is the first step I move.

![](2.jpg)

This is the second step I move.

![](3.jpg)

This is the third step I move.

![](4.jpg)

This is the fourth step I move.

![](5.jpg)

This is the fifth step I move.

![](6.jpg)

This is the sixth step I move.

![](7.jpg)

This is the seventh step I move.

![](8.jpg)

This is the eighth step I move.

![](9.jpg)

This is the final step I move.

Thank you for reading this story.




In [11]:
from pyquery import PyQuery as pq

In [12]:
#r = pq(readable_article)
#r('img').map(lambda i, e: pq(e).attr('src', '%s/%s' % (base_url, pq(e).attr('src'))))

In [13]:
#readable_article

In [14]:
from html.parser import HTMLParser
#from HTMLParser import HTMLParser
#from htmlentitydefs import name2codepoint

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)
        for attr in attrs:
            print("     attr:", attr)
    def handle_endtag(self, tag):
        print("End tag  :", tag)
    def handle_data(self, data):
        print("Data     :", data)
    def handle_comment(self, data):
        print("Comment  :", data)
    def handle_entityref(self, name):
        #c = unichr(name2codepoint[name])
        c = 'entity name should be here'
        print("Named ent:", c)
    def handle_charref(self, name):
        if name.startswith('x'):
            c = unichr(int(name[1:], 16))
        else:
            c = unichr(int(name))
        print("Num ent  :", c)
    def handle_decl(self, data):
        print("Decl     :", data)

parser = MyHTMLParser()

In [15]:
parser.feed(readable_article)

Start tag: html
Start tag: body
Start tag: div
Start tag: article
     attr: ('id', 'main')
Data     : 

  
Start tag: h1
Data     : Walk: A Boring Story
End tag  : h1
Data     : 

  I'm going to tell you a boring story.

  
Start tag: img
     attr: ('src', '1.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the first step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     attr: ('src', '2.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the second step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     attr: ('src', '3.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the third step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     attr: ('src', '4.jpg')
     attr: ('alt', '')
End tag  : img
Data     : 
  
Start tag: p
Data     : 
    This is the fourth step I move.
  
End tag  : p
Data     : 

  
Start tag: img
     a

Remark: not easy to use

In [16]:
# reference:
# http://stackoverflow.com/questions/20590624/python-beautifulsoup-div-text-and-img-attributes-in-correct-order
import bs4
from bs4 import Tag, BeautifulSoup 

class Extractor(object):
    def __init__(self):
        self.cur_text = ''
        self.result = []
    
    def recursive_extract_text_image(self, obj):
        for child in obj.children:
            if isinstance(child, Tag):
                #result.append(child.get('alt', ''))
                self.recursive_extract_text_image(child)
                if child.name == 'img':
                    self.result.append(('text', self.cur_text))
                    self.cur_text = ''
                    self.result.append(('image',
                                   '%s/%s' % (base_url, child['src'])
                                  ))
            else:
                if len(child.strip()) > 0:
                    self.cur_text += ' ' + child.strip() + ' '

    def html_to_asset_list(self, html):
        bs_obj = BeautifulSoup(html, 'html.parser')
        self.recursive_extract_text_image(bs_obj)
        self.result.append(('text', self.cur_text))
        return self.result

result = Extractor().html_to_asset_list(readable_article)
print(result)

[('text', " Walk: A Boring Story  I'm going to tell you a boring story. "), ('image', 'http://project.initiumlab.com/news2video/case1/1.jpg'), ('text', ' This is the first step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/2.jpg'), ('text', ' This is the second step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/3.jpg'), ('text', ' This is the third step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/4.jpg'), ('text', ' This is the fourth step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/5.jpg'), ('text', ' This is the fifth step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/6.jpg'), ('text', ' This is the sixth step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/7.jpg'), ('text', ' This is the seventh step I move. '), ('image', 'http://project.initiumlab.com/news2video/case1/8.jpg'), ('text', ' This is the eighth step I move. '), ('image', 'http

In [17]:
import pandas as pd

In [18]:
df_screenplay = pd.DataFrame(result, columns=['type', 'content'])

In [19]:
import hashlib 
def string2hash(s):
    m = hashlib.sha256()
    m.update(s.encode('utf-8'))
    return m.hexdigest()[:16]
df_screenplay['local_src'] = df_screenplay['content'].apply(lambda x: string2hash(x))

In [20]:
image_selector = (df_screenplay['type'] == 'image')
df_screenplay.loc[image_selector, 'filename'] = df_screenplay.loc[
    image_selector, 'content'].apply(lambda x: path.basename(x))
df_screenplay.loc[image_selector, 'extname'] = df_screenplay.loc[
    image_selector, 'filename'].apply(lambda x: path.splitext(x)[1])
df_screenplay = df_screenplay.fillna('')
df_screenplay['download_name'] = df_screenplay['local_src'] + df_screenplay['extname']
df_screenplay['converted_name'] = df_screenplay['local_src'] + '.png'
df_screenplay

,type,content,local_src,filename,extname,download_name,converted_name
0,text,Walk: A Boring Story I'm going to tell you a...,0b6f2772f708dd15,,,0b6f2772f708dd15,0b6f2772f708dd15.png
1,image,http://project.initiumlab.com/news2video/case1...,5bf67d1c0729bd50,1.jpg,.jpg,5bf67d1c0729bd50.jpg,5bf67d1c0729bd50.png
2,text,This is the first step I move.,e99d8b267c0f2b29,,,e99d8b267c0f2b29,e99d8b267c0f2b29.png
3,image,http://project.initiumlab.com/news2video/case1...,3a9e810ed3b010a4,2.jpg,.jpg,3a9e810ed3b010a4.jpg,3a9e810ed3b010a4.png
4,text,This is the second step I move.,07c025cc4e45e727,,,07c025cc4e45e727,07c025cc4e45e727.png
5,image,http://project.initiumlab.com/news2video/case1...,b30b2910be978147,3.jpg,.jpg,b30b2910be978147.jpg,b30b2910be978147.png
6,text,This is the third step I move.,b30a844bd7d16bd3,,,b30a844bd7d16bd3,b30a844bd7d16bd3.png
7,image,http://project.initiumlab.com/news2video/case1...,f7c4cc87ce36317c,4.jpg,.jpg,f7c4cc87ce36317c.jpg,f7c4cc87ce36317c.png
8,text,This is the fourth step I move.,f5329785a4ee9a50,,,f5329785a4ee9a50,f5329785a4ee9a50.png
9,image,http://project.initiumlab.com/news2video/case1...,bb32e45dfe24271c,5.jpg,.jpg,bb32e45dfe24271c.jpg,bb32e45dfe24271c.png


In [21]:
commands = []
for (i, r) in df_screenplay.iterrows():
    if r['type'] == 'image':
        commands.append('wget {content} -O {download_name}'.format(**r))
        
for (i, r) in df_screenplay.iterrows():
    if r['type'] == 'image':
        commands.append('convert {download_name} {converted_name}'.format(**r))
commands

['wget http://project.initiumlab.com/news2video/case1/1.jpg -O 5bf67d1c0729bd50.jpg',
 'wget http://project.initiumlab.com/news2video/case1/2.jpg -O 3a9e810ed3b010a4.jpg',
 'wget http://project.initiumlab.com/news2video/case1/3.jpg -O b30b2910be978147.jpg',
 'wget http://project.initiumlab.com/news2video/case1/4.jpg -O f7c4cc87ce36317c.jpg',
 'wget http://project.initiumlab.com/news2video/case1/5.jpg -O bb32e45dfe24271c.jpg',
 'wget http://project.initiumlab.com/news2video/case1/6.jpg -O 9126b1c656683fc7.jpg',
 'wget http://project.initiumlab.com/news2video/case1/7.jpg -O 41d9ad7a1b980e63.jpg',
 'wget http://project.initiumlab.com/news2video/case1/8.jpg -O 50caf99e2383f614.jpg',
 'wget http://project.initiumlab.com/news2video/case1/9.jpg -O f30c6fbc6875126a.jpg',
 'convert 5bf67d1c0729bd50.jpg 5bf67d1c0729bd50.png',
 'convert 3a9e810ed3b010a4.jpg 3a9e810ed3b010a4.png',
 'convert b30b2910be978147.jpg b30b2910be978147.png',
 'convert f7c4cc87ce36317c.jpg f7c4cc87ce36317c.png',
 'convert 

In [22]:
import os
for c in commands:
    os.system(c)

In [23]:
commands = []
for (i, r) in df_screenplay.iterrows():
    if r['type'] == 'text':
        commands.append('say --output-file={local_src}.m4a --voice=daniel --rate=150 --progress --file-format=m4af "{content}"'.format(**r))
for (i, r) in df_screenplay.iterrows():
    if r['type'] == 'text':
        commands.append('avconv -i {local_src}.m4a -y {local_src}.wav'.format(**r))
commands

['say --output-file=0b6f2772f708dd15.m4a --voice=daniel --rate=150 --progress --file-format=m4af " Walk: A Boring Story  I\'m going to tell you a boring story. "',
 'say --output-file=e99d8b267c0f2b29.m4a --voice=daniel --rate=150 --progress --file-format=m4af " This is the first step I move. "',
 'say --output-file=07c025cc4e45e727.m4a --voice=daniel --rate=150 --progress --file-format=m4af " This is the second step I move. "',
 'say --output-file=b30a844bd7d16bd3.m4a --voice=daniel --rate=150 --progress --file-format=m4af " This is the third step I move. "',
 'say --output-file=f5329785a4ee9a50.m4a --voice=daniel --rate=150 --progress --file-format=m4af " This is the fourth step I move. "',
 'say --output-file=ac1257d717ef7162.m4a --voice=daniel --rate=150 --progress --file-format=m4af " This is the fifth step I move. "',
 'say --output-file=7539841ff27fa2c9.m4a --voice=daniel --rate=150 --progress --file-format=m4af " This is the sixth step I move. "',
 'say --output-file=bc18b83623

In [24]:
for c in commands:
    os.system(c)

In [25]:
text_selector = (df_screenplay['type'] == 'text')

In [26]:
import sh

In [27]:
def get_audio_length(local_src):
    filename = local_src + '.wav'
    # Caveats: can only deal with < 60s audios
    # | grep Duration | cut -f1 -d, | cut -f4 -d:
    seconds = sh.soxi('-D', filename)
    return seconds.strip()
    
df_screenplay.loc[text_selector, 'duration'] = df_screenplay.loc[text_selector, 'local_src'].apply(get_audio_length)
df_screenplay

,type,content,local_src,filename,extname,download_name,converted_name,duration
0,text,Walk: A Boring Story I'm going to tell you a...,0b6f2772f708dd15,,,0b6f2772f708dd15,0b6f2772f708dd15.png,4.876190
1,image,http://project.initiumlab.com/news2video/case1...,5bf67d1c0729bd50,1.jpg,.jpg,5bf67d1c0729bd50.jpg,5bf67d1c0729bd50.png,NaN
2,text,This is the first step I move.,e99d8b267c0f2b29,,,e99d8b267c0f2b29,e99d8b267c0f2b29.png,2.461315
3,image,http://project.initiumlab.com/news2video/case1...,3a9e810ed3b010a4,2.jpg,.jpg,3a9e810ed3b010a4.jpg,3a9e810ed3b010a4.png,NaN
4,text,This is the second step I move.,07c025cc4e45e727,,,07c025cc4e45e727,07c025cc4e45e727.png,2.507755
5,image,http://project.initiumlab.com/news2video/case1...,b30b2910be978147,3.jpg,.jpg,b30b2910be978147.jpg,b30b2910be978147.png,NaN
6,text,This is the third step I move.,b30a844bd7d16bd3,,,b30a844bd7d16bd3,b30a844bd7d16bd3.png,2.414875
7,image,http://project.initiumlab.com/news2video/case1...,f7c4cc87ce36317c,4.jpg,.jpg,f7c4cc87ce36317c.jpg,f7c4cc87ce36317c.png,NaN
8,text,This is the fourth step I move.,f5329785a4ee9a50,,,f5329785a4ee9a50,f5329785a4ee9a50.png,2.414875
9,image,http://project.initiumlab.com/news2video/case1...,bb32e45dfe24271c,5.jpg,.jpg,bb32e45dfe24271c.jpg,bb32e45dfe24271c.png,NaN


In [28]:
import numpy as np
np.ceil(4.8)

5.0

In [29]:
scenes = []
df_sp_orged = df_screenplay.reset_index()
df_sp_orged['group'] = df_sp_orged['index'].apply(lambda x: int((x + 1) / 2))
for (gname, group) in df_sp_orged.groupby('group'):
    if len(group[group['type'] == 'image']) == 0:
        fn_image = 'default-image.png'
    else:
        fn_image = group[group['type'] == 'image']['converted_name'].values[0]
        
    if len(group[group['type'] == 'text']) == 0:
        duration = 1.53
        fn_audio = 'default-audio.mp4'
    else:
        duration = group[group['type'] == 'text']['duration'].values[0]
        fn_audio = group[group['type'] == 'text']['local_src'].values[0] + '.m4a'
        
    scenes.append(('%04d' % gname, fn_image, duration, fn_audio))

df_scenes = pd.DataFrame(scenes, columns=['group', 'fn_image', 'duration', 'fn_audio'])
df_scenes

,group,fn_image,duration,fn_audio
0,0000,default-image.png,4.876190,0b6f2772f708dd15.m4a
1,0001,5bf67d1c0729bd50.png,2.461315,e99d8b267c0f2b29.m4a
2,0002,3a9e810ed3b010a4.png,2.507755,07c025cc4e45e727.m4a
3,0003,b30b2910be978147.png,2.414875,b30a844bd7d16bd3.m4a
4,0004,f7c4cc87ce36317c.png,2.414875,f5329785a4ee9a50.m4a
5,0005,bb32e45dfe24271c.png,2.461315,ac1257d717ef7162.m4a
6,0006,9126b1c656683fc7.png,2.507755,7539841ff27fa2c9.m4a
7,0007,41d9ad7a1b980e63.png,2.554195,bc18b83623f6c7c4.m4a
8,0008,50caf99e2383f614.png,2.368435,8bd9c4d8f557bc46.m4a
9,0009,f30c6fbc6875126a.png,4.783311,e446517dd3833069.m4a


In [30]:
%%bash
cp default/* .

In [31]:
df_scenes['fn_video_only'] = 'group' + df_scenes['group'] + '.mp4'
df_scenes['fn_video'] = 'group' + df_scenes['group'] + '-a.mp4'
#df_scenes['duration'] = df_scenes['duration'].apply(lambda x: int(np.ceil(float(x))))
df_scenes['fn_image_resized'] = 'resized-' + df_scenes['fn_image']
df_scenes['fn_audio_only'] = 'group' + df_scenes['group'] + '-audio.m4a'
df_scenes

,group,fn_image,duration,fn_audio,fn_video_only,fn_video,fn_image_resized,fn_audio_only
0,0000,default-image.png,4.876190,0b6f2772f708dd15.m4a,group0000.mp4,group0000-a.mp4,resized-default-image.png,group0000-audio.m4a
1,0001,5bf67d1c0729bd50.png,2.461315,e99d8b267c0f2b29.m4a,group0001.mp4,group0001-a.mp4,resized-5bf67d1c0729bd50.png,group0001-audio.m4a
2,0002,3a9e810ed3b010a4.png,2.507755,07c025cc4e45e727.m4a,group0002.mp4,group0002-a.mp4,resized-3a9e810ed3b010a4.png,group0002-audio.m4a
3,0003,b30b2910be978147.png,2.414875,b30a844bd7d16bd3.m4a,group0003.mp4,group0003-a.mp4,resized-b30b2910be978147.png,group0003-audio.m4a
4,0004,f7c4cc87ce36317c.png,2.414875,f5329785a4ee9a50.m4a,group0004.mp4,group0004-a.mp4,resized-f7c4cc87ce36317c.png,group0004-audio.m4a
5,0005,bb32e45dfe24271c.png,2.461315,ac1257d717ef7162.m4a,group0005.mp4,group0005-a.mp4,resized-bb32e45dfe24271c.png,group0005-audio.m4a
6,0006,9126b1c656683fc7.png,2.507755,7539841ff27fa2c9.m4a,group0006.mp4,group0006-a.mp4,resized-9126b1c656683fc7.png,group0006-audio.m4a
7,0007,41d9ad7a1b980e63.png,2.554195,bc18b83623f6c7c4.m4a,group0007.mp4,group0007-a.mp4,resized-41d9ad7a1b980e63.png,group0007-audio.m4a
8,0008,50caf99e2383f614.png,2.368435,8bd9c4d8f557bc46.m4a,group0008.mp4,group0008-a.mp4,resized-50caf99e2383f614.png,group0008-audio.m4a
9,0009,f30c6fbc6875126a.png,4.783311,e446517dd3833069.m4a,group0009.mp4,group0009-a.mp4,resized-f30c6fbc6875126a.png,group0009-audio.m4a


In [32]:
commands = []
for (i, r) in df_scenes.iterrows():
    commands.append('cp {fn_audio} {fn_audio_only}'.format(**r))
commands
for c in commands:
    os.system(c)

In [33]:
commands = []
for (i, r) in df_scenes.iterrows():
    commands.append('convert {fn_image} -resize 600x400! {fn_image_resized}'.format(**r))
commands
for c in commands:
    os.system(c)

In [34]:
commands = []
for (i, r) in df_scenes.iterrows():
    commands.append('ffmpeg -f image2 -r 1/{duration} -i {fn_image_resized} -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 {fn_video_only}'.format(**r))
for (i, r) in df_scenes.iterrows():
    commands.append('ffmpeg -i {fn_video_only} -i {fn_audio} -qscale:v 1 -copyts -vcodec copy -acodec copy -y {fn_video}'.format(**r))
    #commands.append('ffmpeg -i {fn_video_only} -i {fn_audio} -map 0:0 -map 1 -vcodec copy -acodec copy -y {fn_video}'.format(**r))
commands

['ffmpeg -f image2 -r 1/4.876190 -i resized-default-image.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0000.mp4',
 'ffmpeg -f image2 -r 1/2.461315 -i resized-5bf67d1c0729bd50.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0001.mp4',
 'ffmpeg -f image2 -r 1/2.507755 -i resized-3a9e810ed3b010a4.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0002.mp4',
 'ffmpeg -f image2 -r 1/2.414875 -i resized-b30b2910be978147.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0003.mp4',
 'ffmpeg -f image2 -r 1/2.414875 -i resized-f7c4cc87ce36317c.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0004.mp4',
 'ffmpeg -f image2 -r 1/2.461315 -i resized-bb32e45dfe24271c.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0005.mp4',
 'ffmpeg -f image2 -r 1/2.507755 -i resized-9126b1c656683fc7.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0006.mp4',
 'ffmpeg -f image2 -r 1/2.554195 -i resized-41d9ad7a1b980e63.png -qscale:v 1 -copyts -vcodec mpeg4 -y -r 25 group0007.mp4',
 'ffmpeg -f

In [35]:
for c in commands:
    os.system(c)

In [36]:
open('playlist.txt', 'w').write('\n'.join(list(df_scenes['fn_video'].apply(lambda x: "file '%s'" % x))))

229

In [37]:
os.system('ffmpeg -f concat -i playlist.txt -c copy -y output.mp4')
#os.system('ffmpeg -i output.mpg -qscale:v 2 -copyts output.mp4')

0

First time I reach here, we have DTS problem:

```
[concat @ 0x7f9e02805800] DTS 162144 < 181089 out of order
[mp4 @ 0x7f9e02825200] Non-monotonous DTS in output stream 0:0; previous: 181089, current: 162144; changing to 181090. This may result in incorrect timestamps in the output file.
[mp4 @ 0x7f9e02825200] Non-monotonous DTS in output stream 0:0; previous: 202320, current: 156516; changing to 202321. This may result in incorrect timestamps in the output file.
[mp4 @ 0x7f9e02825200] Non-monotonous DTS in output stream 0:0; previous: 481191, current: 433383; changing to 481192. This may result in incorrect timestamps in the output file.
[mp4 @ 0x7f9e02825200] Non-monotonous DTS in output stream 0:0; previous: 481192, current: 341667; changing to 481193. This may result in incorrect timestamps in the output file.
```

some info:
http://dranger.com/ffmpeg/tutorial05.html

This playlist yields valid graphics:

```
%cat playlist.txt
#file 'group0000.mp4'
file 'group0001.mp4'
file 'group0002.mp4'
file 'group0003.mp4'
file 'group0004.mp4'
file 'group0005.mp4'
file 'group0006.mp4'
file 'group0007.mp4'
file 'group0008.mp4'
file 'group0009.mp4'
```

As suggested by http://stackoverflow.com/questions/12847262/ffmpeg-video-concatenation-audio-videotrack-synchronisation , we add `-copyts` to every command. 
The result is a almost aligned video with some sound missing.
(not working)

someone else mentioned `-sameq` option. 
http://stackoverflow.com/questions/7388184/ffmpeg-sound-going-out-of-sync-with-concat-or-ss
(not tested)

This post
http://stackoverflow.com/questions/7015489/building-video-from-images-with-ffmpeg-w-pause-between-frames
inspires me to try `-r 25` for the output stream. That seems to solve the out-of-sync problem.
The trap here: `ffmpeg` accepts `-r` at two positions, one for input stream, one for output stream. We need to set the output stream to something accepted by the standard.